In [1]:
import datacache

In [2]:
cache = datacache.Cache("weirdo")

In [3]:
path = cache.fetch("http://ftp.ensembl.org/pub/release-108/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz")

In [4]:
path

'/Users/iskander/Library/Caches/weirdo/4941639f3ec64d7154f6aae21ee97c1c.http___ftp.ensembl.org_pub_release-108_fasta_homo_sapiens_pep_Homo_sapiens.GRCh38.pep.all.fa.gz'

In [5]:
import datacache
import gzip

def read_protein_sequence_from_path(path, max_count=None):
    proteins = {}
    buffer = []
    last_name = None
    count = 0 
    def add_entry():
        nonlocal count 
        if buffer and last_name and (max_count is None or count < max_count):
            seq = "".join(buffer)
            if "*" not in seq:
                proteins[last_name] = seq
                count += 1

            buffer.clear()
            
    with gzip.open(path, "rt", newline="\n") as f:
        for l in f:
            if l[0] == ">":
                add_entry()
                if max_count is not None and count >= max_count:
                    break
                last_name = l[1:l.find(" ")]
            elif len(l) == 0:
                continue
            else:
                # get rid of last line because it's '\n'
                buffer.append(l[:-1])
    add_entry()
    return proteins

def read_protein_sequence_from_url(url, max_count=None):
    path = cache.fetch(url)
    return read_protein_sequence_from_path(path, max_count=max_count)

In [6]:
%time proteins = read_protein_sequence_from_url("http://ftp.ensembl.org/pub/release-108/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz")

CPU times: user 508 ms, sys: 24.9 ms, total: 533 ms
Wall time: 565 ms


In [7]:
proteins

{'ENSP00000488240.1': 'GTGG',
 'ENSP00000451042.1': 'EI',
 'ENSP00000452494.1': 'TGGY',
 'ENSP00000451515.1': 'PSY',
 'ENSP00000487941.1': 'GTGG',
 'ENSP00000488000.1': 'GIVGAT',
 'ENSP00000488695.1': 'LTG',
 'ENSP00000487903.1': 'VLRFLEWLLY',
 'ENSP00000487789.1': 'GYSSGWY',
 'ENSP00000488522.1': 'GITGT',
 'ENSP00000488392.1': 'GYSSGY',
 'ENSP00000488589.1': 'EYSSSS',
 'ENSP00000487922.1': 'VDIVATI',
 'ENSP00000487775.1': 'VLRYFDWLL',
 'ENSP00000487812.1': 'GITGT',
 'ENSP00000488113.1': 'VEMATI',
 'ENSP00000487937.1': 'VDTAMV',
 'ENSP00000488475.1': 'VLLWFGELL',
 'ENSP00000488840.1': 'GTTGT',
 'ENSP00000488201.1': 'VDTAMV',
 'ENSP00000488592.1': 'GYSSSWY',
 'ENSP00000488720.1': 'GITGT',
 'ENSP00000473787.1': 'GITGT',
 'ENSP00000473849.1': 'VDIVSTI',
 'ENSP00000473700.1': 'VDIVSTI',
 'ENSP00000474222.1': 'GITGT',
 'ENSP00000418639.1': 'LTG',
 'ENSP00000420733.1': 'GIVGAT',
 'ENSP00000417751.1': 'GYSSGY',
 'ENSP00000419139.1': 'VEMATI',
 'ENSP00000420556.1': 'GITGT',
 'ENSP00000418010.1

In [8]:
import shellinford
fm = shellinford.FMIndex()

In [9]:
%time fm.build(proteins)

CPU times: user 2min 29s, sys: 1.36 s, total: 2min 30s
Wall time: 2min 34s


In [10]:
proteins

{'ENSP00000488240.1': 'GTGG',
 'ENSP00000451042.1': 'EI',
 'ENSP00000452494.1': 'TGGY',
 'ENSP00000451515.1': 'PSY',
 'ENSP00000487941.1': 'GTGG',
 'ENSP00000488000.1': 'GIVGAT',
 'ENSP00000488695.1': 'LTG',
 'ENSP00000487903.1': 'VLRFLEWLLY',
 'ENSP00000487789.1': 'GYSSGWY',
 'ENSP00000488522.1': 'GITGT',
 'ENSP00000488392.1': 'GYSSGY',
 'ENSP00000488589.1': 'EYSSSS',
 'ENSP00000487922.1': 'VDIVATI',
 'ENSP00000487775.1': 'VLRYFDWLL',
 'ENSP00000487812.1': 'GITGT',
 'ENSP00000488113.1': 'VEMATI',
 'ENSP00000487937.1': 'VDTAMV',
 'ENSP00000488475.1': 'VLLWFGELL',
 'ENSP00000488840.1': 'GTTGT',
 'ENSP00000488201.1': 'VDTAMV',
 'ENSP00000488592.1': 'GYSSSWY',
 'ENSP00000488720.1': 'GITGT',
 'ENSP00000473787.1': 'GITGT',
 'ENSP00000473849.1': 'VDIVSTI',
 'ENSP00000473700.1': 'VDIVSTI',
 'ENSP00000474222.1': 'GITGT',
 'ENSP00000418639.1': 'LTG',
 'ENSP00000420733.1': 'GIVGAT',
 'ENSP00000417751.1': 'GYSSGY',
 'ENSP00000419139.1': 'VEMATI',
 'ENSP00000420556.1': 'GITGT',
 'ENSP00000418010.1

In [21]:
def build_kmer_index(seqs, min_k=8, max_k=15):
    from collections import Counter
    kmer_counts = Counter()
    for (name, seq) in seqs.items():
        n = len(seq)
        
        # if n < min_k:
        #    continue
        for k in range(min_k, max_k + 1):
            # if n < k:
            #    continue
            for i in range(n - k + 1):
                kmer_counts[seq[i:i+k]] += 1
    return kmer_counts


In [22]:
%time kmers = build_kmer_index(proteins)

CPU times: user 2min 29s, sys: 7.56 s, total: 2min 36s
Wall time: 2min 45s


KeyboardInterrupt: 